In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Load dataset

In [2]:
mnist = input_data.read_data_sets("MNIST_data")

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
train_data = mnist.train.images
train_labels = mnist.train.labels

validation_data = mnist.validation.images
validation_labels = mnist.validation.labels

test_data = mnist.test.images
test_labels = mnist.test.labels

print("train_data", train_data.shape, "train_labels", train_labels.shape)
print("validation_data", validation_data.shape, "validation_labels", validation_labels.shape)
print("test_data", test_data.shape, "test_labels", test_labels.shape)

train_data (55000, 784) train_labels (55000,)
validation_data (5000, 784) validation_labels (5000,)
test_data (10000, 784) test_labels (10000,)


# Create input functions for training, validation and testing

In [4]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=64,
    num_epochs=10,
    shuffle=True
)

validation_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": validation_data},
    y=validation_labels,
    num_epochs=1,
    shuffle=False)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_data},
    y=test_labels,
    num_epochs=1,
    shuffle=False)

# Create model function

In [5]:
def my_model_fn(features, labels, mode):
    """Model function for our CNN"""
    
    net = tf.reshape(features['x'], [-1, 28, 28, 1])
    for _ in range(3):
        net = tf.layers.conv2d(
            inputs=net,
            filters=32,
            kernel_size=[3, 3],
            padding="same",
            activation=tf.nn.relu
        )
        net = tf.layers.max_pooling2d(
            inputs=net,
            pool_size=[2, 2],
            strides=[2,2]
        )
    net = tf.layers.flatten(net)
    net = tf.layers.dense(inputs=net, units=64)
    logits = tf.layers.dense(inputs=net, units=10)
    
    predictions = {
        'classes': tf.argmax(input=logits, axis=1),
        'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:    
        export_outputs = {
            'predictions': tf.estimator.export.PredictOutput(predictions)
        }
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
     

    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, 
                                           logits=logits)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    

    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions['classes'])
    }
    return tf.estimator.EstimatorSpec(mode=mode, 
                                      loss=loss, 
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)

# Create estimator

In [6]:
mnist_estimator = tf.estimator.Estimator(
    model_fn=my_model_fn,
    model_dir="E:\\temp\\mnist_estimator"
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000222A39FDBA8>, '_task_id': 0, '_save_checkpoints_secs': 600, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'E:\\temp\\mnist_estimator', '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_num_worker_replicas': 1, '_is_chief': True, '_task_type': 'worker', '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_service': None}


# Train and evaluate

In [7]:
train_spec = tf.estimator.TrainSpec(
    input_fn=train_input_fn,
    max_steps=20000,
)

validation_spec = tf.estimator.EvalSpec(
    input_fn=validation_input_fn,
    steps=50, throttle_secs=60
)


tf.estimator.train_and_evaluate(mnist_estimator, train_spec, validation_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into E:\temp\mnist_estimator\model.ckpt.
INFO:tensorflow:step = 1, loss = 2.26988
INFO:tensorflow:global_step/sec: 204.793
INFO:tensorflow:step = 101, loss = 2.30639 (0.505 sec)
INFO:tensorflow:global_step/sec: 230.581
INFO:tensorflow:step = 201, loss = 2.24275 (0.416 sec)
INFO:tensorflow:global_step/sec: 259.724
INFO:tensorflow:step = 301, loss = 2.26652 (0.384 sec)
INFO:tensorflow:global_step/sec: 255.729
INFO:tensorflow:step = 401, loss = 2.26944 (0.391 sec)
INFO:tensorflow:global_step/sec: 264.527
INFO:tensorflow:step = 501, loss = 2.26268 (0.379 sec)
INFO:tensorflow:global_step/sec: 265.2
INFO:tensorflow:step = 601, loss = 2.24345 (0.376 sec)
INFO:tensorflow:global_step/sec: 264.51
INFO:te

INFO:tensorflow:step = 8101, loss = 0.201857 (0.381 sec)
INFO:tensorflow:global_step/sec: 257.711
INFO:tensorflow:step = 8201, loss = 0.127826 (0.388 sec)
INFO:tensorflow:global_step/sec: 257.047
INFO:tensorflow:step = 8301, loss = 0.276485 (0.389 sec)
INFO:tensorflow:global_step/sec: 267.352
INFO:tensorflow:step = 8401, loss = 0.17491 (0.374 sec)
INFO:tensorflow:global_step/sec: 256.385
INFO:tensorflow:step = 8501, loss = 0.136913 (0.390 sec)
INFO:tensorflow:Saving checkpoints for 8594 into E:\temp\mnist_estimator\model.ckpt.
INFO:tensorflow:Loss for final step: 0.177838.
INFO:tensorflow:Starting evaluation at 2018-01-21-16:52:26
INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-8594
INFO:tensorflow:Evaluation [1/50]
INFO:tensorflow:Evaluation [2/50]
INFO:tensorflow:Evaluation [3/50]
INFO:tensorflow:Evaluation [4/50]
INFO:tensorflow:Evaluation [5/50]
INFO:tensorflow:Evaluation [6/50]
INFO:tensorflow:Evaluation [7/50]
INFO:tensorflow:Evaluation [8/50]
INFO:te

INFO:tensorflow:step = 14395, loss = 0.142185 (0.380 sec)
INFO:tensorflow:global_step/sec: 259.709
INFO:tensorflow:step = 14495, loss = 0.0895127 (0.385 sec)
INFO:tensorflow:global_step/sec: 262.457
INFO:tensorflow:step = 14595, loss = 0.143253 (0.381 sec)
INFO:tensorflow:global_step/sec: 263.848
INFO:tensorflow:step = 14695, loss = 0.199772 (0.380 sec)
INFO:tensorflow:global_step/sec: 256.382
INFO:tensorflow:step = 14795, loss = 0.165248 (0.389 sec)
INFO:tensorflow:global_step/sec: 266.636
INFO:tensorflow:step = 14895, loss = 0.0609644 (0.376 sec)
INFO:tensorflow:global_step/sec: 249.955
INFO:tensorflow:step = 14995, loss = 0.203859 (0.399 sec)
INFO:tensorflow:global_step/sec: 261.039
INFO:tensorflow:step = 15095, loss = 0.21164 (0.385 sec)
INFO:tensorflow:global_step/sec: 258.382
INFO:tensorflow:step = 15195, loss = 0.302156 (0.385 sec)
INFO:tensorflow:global_step/sec: 255.729
INFO:tensorflow:step = 15295, loss = 0.262043 (0.391 sec)
INFO:tensorflow:global_step/sec: 255.074
INFO:tens

INFO:tensorflow:Evaluation [11/50]
INFO:tensorflow:Evaluation [12/50]
INFO:tensorflow:Evaluation [13/50]
INFO:tensorflow:Evaluation [14/50]
INFO:tensorflow:Evaluation [15/50]
INFO:tensorflow:Evaluation [16/50]
INFO:tensorflow:Evaluation [17/50]
INFO:tensorflow:Evaluation [18/50]
INFO:tensorflow:Evaluation [19/50]
INFO:tensorflow:Evaluation [20/50]
INFO:tensorflow:Evaluation [21/50]
INFO:tensorflow:Evaluation [22/50]
INFO:tensorflow:Evaluation [23/50]
INFO:tensorflow:Evaluation [24/50]
INFO:tensorflow:Evaluation [25/50]
INFO:tensorflow:Evaluation [26/50]
INFO:tensorflow:Evaluation [27/50]
INFO:tensorflow:Evaluation [28/50]
INFO:tensorflow:Evaluation [29/50]
INFO:tensorflow:Evaluation [30/50]
INFO:tensorflow:Evaluation [31/50]
INFO:tensorflow:Evaluation [32/50]
INFO:tensorflow:Evaluation [33/50]
INFO:tensorflow:Evaluation [34/50]
INFO:tensorflow:Evaluation [35/50]
INFO:tensorflow:Evaluation [36/50]
INFO:tensorflow:Evaluation [37/50]
INFO:tensorflow:Evaluation [38/50]
INFO:tensorflow:Eval

# Perform evaluation on the test set

In [8]:
mnist_estimator.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2018-01-21-16:53:20
INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-20000
INFO:tensorflow:Finished evaluation at 2018-01-21-16:53:20
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9656, global_step = 20000, loss = 0.112436


{'accuracy': 0.96560001, 'global_step': 20000, 'loss': 0.112436}

# Getting the result of each prediction

In [9]:
test_results = mnist_estimator.predict(input_fn=test_input_fn)
print(test_results)

<generator object Estimator.predict at 0x00000222A5CF7D00>


In [10]:
import itertools

In [11]:
list(itertools.islice(test_results, 3))

INFO:tensorflow:Restoring parameters from E:\temp\mnist_estimator\model.ckpt-20000


[{'classes': 7,
  'probabilities': array([  2.46418438e-07,   3.63487857e-07,   2.92197728e-05,
           2.04831464e-04,   8.29491853e-09,   1.75949182e-07,
           9.80678871e-11,   9.99503613e-01,   1.90804872e-06,
           2.59665190e-04], dtype=float32)},
 {'classes': 2,
  'probabilities': array([  1.44028472e-05,   3.06248803e-05,   9.99740660e-01,
           1.37188845e-05,   8.96609783e-12,   4.97646340e-07,
           1.57032773e-04,   2.98117864e-09,   4.29810025e-05,
           5.71593077e-12], dtype=float32)},
 {'classes': 1,
  'probabilities': array([  8.74112538e-06,   9.98622775e-01,   1.96440378e-04,
           4.93280313e-05,   2.42024311e-04,   7.93635263e-06,
           4.97537141e-04,   1.50834690e-04,   1.74942266e-04,
           4.95492022e-05], dtype=float32)}]